In [1]:
# Imports
import os, pytz
from django.apps import apps as django_apps
from edc_visit_schedule.site_visit_schedules import site_visit_schedules

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

tz = pytz.timezone('Africa/Gaborone')

subject_identifier = 'B142-040990712-7'

In [2]:
delv_visit = MaternalVisit.objects.get(subject_identifier=subject_identifier, visit_code='2000D')
report_datetime = delv_visit.report_datetime
report_datetime = report_datetime.astimezone(tz)

onsch = OnScheduleCohortAQuarterly.objects.get(subject_identifier=subject_identifier)
ssh = SubjectScheduleHistory.objects.get(
    subject_identifier=subject_identifier, schedule_name=onsch.schedule_name)
ssh.delete()
onsch.delete()

_, schedule = site_visit_schedules.get_by_onschedule_model_schedule_name(
    onschedule_model=ssh.onschedule_model, name=ssh.schedule_name)
schedule.put_on_schedule(
    subject_identifier=onsch.subject_identifier, onschedule_datetime=report_datetime,
    schedule_name=onsch.schedule_name, base_appt_datetime=report_datetime)

onsch = OnScheduleCohortAQuarterly.objects.get(subject_identifier=subject_identifier)
onsch.child_subject_identifier = 'B142-040990712-7-10'
onsch.save()

In [18]:
from django.db import IntegrityError
from edc_appointment.models import Appointment

lower_visit = 2001

visits = MaternalVisit.objects.filter(
    subject_identifier=subject_identifier, schedule_name='a_quarterly1_schedule1')
for visit in visits:
    visit_code = 2001
    while visit_code >= lower_visit:
        try:
            appt = Appointment.objects.get(subject_identifier=subject_identifier,
                                           visit_code=f'{visit_code}M')
        except Appointment.DoesNotExist:
            print('here')
            break
        else:
            earliest_start = (appt.timepoint_datetime -
                               appt.visits.get(appt.visit_code).rlower).astimezone(tz)
    
            latest_start = (appt.timepoint_datetime +
                            appt.visits.get(appt.visit_code).rupper).astimezone(tz)
            visit_dt = visit.report_datetime.astimezone(tz)

            if visit_dt >= earliest_start and visit_dt <= latest_start:
                if visit.reason == 'missed' and visit.visit_code != appt.visit_code:
                    visit.delete()
                else:
                    print(visit.visit_code, earliest_start, visit_dt, latest_start, appt.visit_code)
                    try:
                        visit.appointment = appt
                        visit.save_base(raw=True)
                        appt.appt_datetime = visit_dt
                        appt.save()
                    except IntegrityError:
                        print(visit.visit_code, 'failing..')
                        break
                    else:
                        print(visit.visit_code, 'updated.')

            visit_code += 1

2002M 2023-06-16 16:10:34+02:00 2023-08-03 09:29:09+02:00 2023-09-13 16:10:34+02:00 2004M
2002M updated.
here
2005M 2023-09-14 16:10:34+02:00 2023-11-20 18:30:27+02:00 2023-12-12 16:10:34+02:00 2005M
2005M updated.
here
2006M 2023-12-13 16:10:34+02:00 2023-12-22 12:18:51+02:00 2024-03-11 16:10:34+02:00 2006M
2006M updated.
here
2007M 2024-03-12 16:10:34+02:00 2024-04-17 13:07:16+02:00 2024-06-09 16:10:34+02:00 2007M
2007M updated.
here
2008M 2024-06-10 16:10:34+02:00 2024-08-13 09:23:41+02:00 2024-09-07 16:10:34+02:00 2008M
2008M updated.
here


In [20]:
CrfMetadata.objects.filter(subject_identifier=subject_identifier, visit_code='2002M').delete()

(20, {'edc_metadata.CrfMetadata': 20})

In [21]:
CrfMetadata.objects.filter(subject_identifier=subject_identifier, visit_code='2003M').delete()

(20, {'edc_metadata.CrfMetadata': 20})